# Revised version of PPR1 code

#### Precisely, the code was used to subset flight lines (FL) that have count maximum number of plots to be used later to map biomass and show in PPR1 without any visual arefacts being displayed

In [6]:
import os, sys, pathlib
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, box, mapping
from matplotlib import pyplot as plt
from matplotlib.colors import TwoSlopeNorm
from matplotlib.gridspec import GridSpec
from matplotlib.font_manager import findfont, FontProperties
import numpy as np
from geopandas.io.file import _to_file
import fiona
import rasterio
from rasterio.plot import show
from rasterio.mask import mask
from rasterio.enums import Resampling
from rasterio.warp import reproject, calculate_default_transform
from rasterio.crs import CRS
from rasterio.windows import Window
from tqdm import tqdm
import glob
import re
import contextily as ctx
from pathlib import Path


In [7]:
print("CWD:", os.getcwd())

CWD: C:\Users\NargizSafaraliyeva\OneDrive - geo.uzh.ch\Desktop\2nd-paper\phd_ppr2


## LAMBERTIAN data sources

In [ ]:
# 2010
# FL_42_2010_LAM = r"D:/.../L_2010_APEX_processed2011/TRU_subset/CHNP1100624_042e_polishRefl_sub.bsq"
FL_52_2010_LAM = r"D:/.../L_2010_APEX_processed2011/TRU_subset/CHNP1100624_052e_polishRefl_sub.bsq"
# FL_62_2010_LAM = r"D:/.../L_2010_APEX_processed2011/TRU_subset/CHNP1100624_062e_polishRefl_sub.bsq"
# FL_72_2010_LAM = r"D:/.../L_2010_APEX_processed2011/TRU_subset/CHNP1100624_072e_polishRefl_sub.bsq"

# 2012
# FL_12_S1_2012_LAM = r"D:/.../L_2012_APEX/CHNP2120629_a12d_polish_cube000_geo_s1.bsq" #v
FL_32_S1_2012_LAM = r"D:/.../L_2012_APEX/CHNP2120629_a32d_polish_cube000_geo_s1.bsq" #v
# FL_42_S1_2012_LAM = r"D:/.../L_2012_APEX/CHNP2120629_a42d_polish_cube000_geo_s1.bsq" #v
# FL_12_S2_2012_LAM = r"D:/.../L_2012_APEX/CHNP2120629_a12d_polish_cube000_geo_s2.bsq"
# FL_32_S2_2012_LAM = r"D:/.../L_2012_APEX/CHNP2120629_a32d_polish_cube000_geo_s2.bsq"
# FL_42_S2_2012_LAM = r"D:/.../L_2012_APEX/CHNP2120629_a42d_polish_cube000_geo_s2.bsq"

# 2013
# FL_12_S1_2013_LAM  = r"D:/.../L_2013_APEX/M0038130712_a012e_reflct_cube000_geo.bsq"
FL_32_S1_2013_LAM  = r"D:/.../L_2013_APEX/M0038130712_a032e_reflct_cube000_geo.bsq"
# FL_52_S1_2013_LAM  = r"D:/.../L_2013_APEX/M0038130712_a052e_reflct_cube000_geo.bsq"
# FL_12_S2_2013_LAM  = r"D:/.../L_2013_APEX/M0039130712_a012e_reflct_cube000_geo.bsq" # not in ROI
# FL_32_S2_2013_LAM  = r"D:/.../L_2013_APEX/M0039130712_a032e_reflct_cube000_geo.bsq" # not in ROI
# FL_92_2013_LAM     = r"D:/.../L_2013_APEX/M0039130712_a092e_reflct_cube000_geo.bsq" # not in ROI
# FL_112_S3_2013_LAM = r"D:/.../L_2013_APEX/M0039130712_a112e_reflct_cube000_geo.bsq" # not in ROI

## MINNAERT data sources

In [ ]:
# 2010
# FL_42_2010_MIN  = r"D:/.../MM010_CHNP__100624_a042e_reflct_cube000_smcorr_geo.bsq"
FL_52_2010_MIN  = r"D:/.../MM010_CHNP__100624_a052e_reflct_cube000_smcorr_geo.bsq"
# FL_72_2010_MIN  = r"D:/.../MM010_CHNP__100624_a072e_reflct_cube000_smcorr_geo.bsq"

# 2012
# FL_12_2012_MIN  = r"D:/.../SNP_2012/MM047_CHNP__120629_a012e_reflct_cube000_geo.bsq"
FL_22_2012_MIN  = r"D:/.../SNP_2012/MM047_CHNP__120629_a022e_reflct_cube000_geo.bsq"
# FL_32_2012_MIN  = r"D:/.../SNP_2012/MM047_CHNP__120629_a032e_reflct_cube000_geo.bsq"

# 2013
# FL_65_52_2013_MIN  = r"D:/.../SNP_2013/MM065_CHNP__130712_a052e_reflct_cube000_geo.bsq" # v
# FL_65_42_2013_MIN  = r"D:/.../SNP_2013/MM065_CHNP__130712_a042e_reflct_cube000_geo.bsq" # v
# FL_65_12_2013_MIN  = r"D:/.../SNP_2013/MM065_CHNP__130712_a012e_reflct_cube000_geo.bsq" # v
# FL_62_2013_MIN  = r"D:/.../SNP_2013/MM064_CHNP__130712_a062e_reflct_cube000_geo.bsq" # v
# FL_52_2013_MIN  = r"D:/.../SNP_2013/MM064_CHNP__130712_a052e_reflct_cube000_geo.bsq" # v
# FL_42_2013_MIN  = r"D:/.../SNP_2013/MM064_CHNP__130712_a042e_reflct_cube000_geo.bsq" # v
# FL_32_2013_MIN  = r"D:/.../SNP_2013/MM064_CHNP__130712_a032e_reflct_cube000_geo.bsq" # v
FL_22_2013_MIN  = r"D:/.../SNP_2013/MM064_CHNP__130712_a022e_reflct_cube000_geo.bsq" # v
# FL_65_62_2013_MIN  = r"D:/.../SNP_2013/MM065_CHNP__130712_a062e_reflct_cube000_geo.bsq" # not in ROI
# FL_65_32_2013_MIN  = r"D:/.../SNP_2013/MM065_CHNP__130712_a032e_reflct_cube000_geo.bsq" # not in ROI
# FL_65_22_2013_MIN  = r"D:/.../SNP_2013/MM065_CHNP__130712_a022e_reflct_cube000_geo.bsq" # not in ROI

## modified MINNAERT data sources

In [ ]:
# 2010
# FL_42_2010_MOD  = r"D:/.../2010/MM010_CHNP__100624_a042e_reflct_cube000_smcorr_geo.bsq"
FL_52_2010_MOD  = r"D:/.../2010/MM010_CHNP__100624_a052e_reflct_cube000_smcorr_geo.bsq"
# FL_72_2010_MOD  = r"D:/.../2010/MM010_CHNP__100624_a072e_reflct_cube000_smcorr_geo.bsq"

# 2012
# FL_12_2012_MOD  = r"D:/.../2012/MM047_CHNP__120629_a012e_reflct_cube000_smcorr_geo.bsq"
FL_22_2012_MOD  = r"D:/.../2012/MM047_CHNP__120629_a022e_reflct_cube000_smcorr_geo.bsq"
# FL_32_2012_MOD  = r"D:/.../2012/MM047_CHNP__120629_a032e_reflct_cube000_smcorr_geo.bsq"

# 2013
# FL_42_2013_MOD  = r"D:/.../2013/MM064_CHNP__130712_a042e_reflct_cube000_smcorr_geo.bsq"
# FL_32_2013_MOD  = r"D:/.../2013/MM064_CHNP__130712_a032e_reflct_cube000_smcorr_geo.bsq"
FL_22_2013_MOD  = r"D:/.../2013/MM064_CHNP__130712_a022e_reflct_cube000_smcorr_geo.bsq"

In [23]:
# --- Minnaert & Modified Minnaert: CLIP ONLY (EPSG:2056, BSQ) — NO MOSAIC ---
import os, time, math
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.crs import CRS
from rasterio.enums import Resampling
from rasterio.transform import from_origin
from rasterio.warp import reproject
from rasterio.features import geometry_mask
from shapely.geometry import box
from pyproj import Transformer
from tqdm import tqdm
import matplotlib.pyplot as plt
from datetime import datetime

# ===================== USER: AOI rectangle (EPSG:2056) =====================
AOI_VECTOR = r"D:/.../L_2010_APEX_processed2011/TRU_subset/mosaic_extent_rect_EPSG2056.shp"

# ===================== USER: FLIGHTLINE PATHS (UNCOMMENTED ONLY) =====================
# Make CFG UNIFORM: method -> year -> list of files
CFG = {
    "Lambertian": {
        2010: [
            r"D:/.../L_2010_APEX_processed2011/TRU_subset/CHNP1100624_052e_polishRefl_sub.bsq",
        ],
        2012: [
            r"D:/.../L_2012_APEX/CHNP2120629_a32d_polish_cube000_geo_s1.bsq",
        ],
        2013: [
            r"D:/.../L_2013_APEX/M0038130712_a032e_reflct_cube000_geo.bsq",
        ],
    },
    # "Minnaert": {
    #     2010: [
    #         r"D:/.../MM010_CHNP__100624_a052e_reflct_cube000_smcorr_geo.bsq",
    #     ],
    #     2012: [
    #         r"D:/.../SNP_2012/MM047_CHNP__120629_a022e_reflct_cube000_geo.bsq",
    #     ],
    #     2013: [
    #         r"D:/.../SNP_2013/MM064_CHNP__130712_a022e_reflct_cube000_geo.bsq",
    #     ],
    # },
    # "ModMinnaert": {
    #     2010: [
    #         r"D:/.../2010/MM010_CHNP__100624_a052e_reflct_cube000_smcorr_geo.bsq",
    #     ],
    #     2012: [
    #         r"D:/.../2012/MM047_CHNP__120629_a022e_reflct_cube000_smcorr_geo.bsq",
    #     ],
    #     2013: [
    #         r"D:/.../2013/MM064_CHNP__130712_a022e_reflct_cube000_smcorr_geo.bsq",
    #     ],
    # },
}

# ===================== CONSTANTS =====================
TARGET_CRS = CRS.from_epsg(2056)     # LV95
ASSUMED_LV03 = CRS.from_epsg(21781)  # LV03 fallback for LOCAL/Arbitrary
RESAMPLING = Resampling.nearest      # reflectance-safe

def stamp(msg): print(f"[{datetime.now().strftime('%H:%M:%S')}] {msg}")

# ===================== HELPERS =====================
def detect_effective_src_crs(ds):
    """Trusted CRS; if authority missing, infer 2056 vs 21781 from bounds; default LV03."""
    if ds.crs and ds.crs.to_authority():
        return ds.crs, "[as-is]"
    L, B, R, T = ds.bounds
    mx = (L + R) / 2.0; my = (B + T) / 2.0
    if (2.0e6 <= abs(mx) <= 3.5e6) and (7.0e5 <= abs(my) <= 2.0e6):
        return CRS.from_epsg(2056), "[assumed: LV95 from bounds]"
    return CRS.from_epsg(21781), "[assumed: LV03 default]"

def overlaps_aoi_in_2056(src_bounds, src_crs, aoi_bounds_2056):
    L, B, R, T = src_bounds
    to_2056 = Transformer.from_crs(src_crs, 2056, always_xy=True).transform
    x0, y0 = to_2056(L, B); x1, y1 = to_2056(L, T); x2, y2 = to_2056(R, B); x3, y3 = to_2056(R, T)
    fl_minx = min(x0, x1, x2, x3); fl_maxx = max(x0, x1, x2, x3)
    fl_miny = min(y0, y1, y2, y3); fl_maxy = max(y0, y1, y2, y3)
    return box(fl_minx, fl_miny, fl_maxx, fl_maxy).intersects(box(*aoi_bounds_2056))

def build_aoi_grid(aoi_path, ref_path):
    aoi = gpd.read_file(aoi_path)
    if aoi.crs is None or aoi.crs.to_epsg() != 2056:
        aoi = aoi.to_crs(2056)
    geom = aoi.geometry.unary_union
    minx, miny, maxx, maxy = geom.bounds
    with rasterio.open(ref_path) as ref:
        _crs, _ = detect_effective_src_crs(ref)
        resx = abs(ref.transform.a); resy = abs(ref.transform.e)
    left   = math.floor(minx / resx) * resx
    right  = math.ceil (maxx / resx) * resx
    bottom = math.floor(miny / resy) * resy
    top    = math.ceil (maxy / resy) * resy
    W = int(round((right - left) / resx))
    H = int(round((top - bottom) / resy))
    transform = from_origin(left, top, resx, resy)
    return geom, transform, W, H, resx, resy

def norm(a):
    a_min, a_max = np.nanmin(a), np.nanmax(a)
    if not np.isfinite(a_min) or not np.isfinite(a_max) or a_max == a_min:
        return np.zeros_like(a, dtype=float)
    return (a - a_min) / (a_max - a_min)

def cleanup_sidecars(data_path):
    """Remove data + ENVI/GDAL sidecars to prevent header-size mismatches."""
    base, ext = os.path.splitext(data_path)
    for p in [data_path, data_path + ".hdr", base + ".hdr", data_path + ".aux.xml"]:
        if os.path.exists(p):
            try: os.remove(p)
            except PermissionError: raise RuntimeError(f"Locked: {p}")

# ===================== CORE =====================
def process_one_set(label, year, paths, aoi_path=AOI_VECTOR):
    """
    Writes ONLY clipped BSQs (per flightline), NO mosaic.
    label: 'Minnaert' / 'ModMinnaert'; year: 2010/2012/2013; paths: list of BSQs.
    """
    print("\n" + "="*90)
    print(f"{label} — {year} (CLIP ONLY, NO MOSAIC)")
    print("="*90)

    if not paths:
        print("No files listed, skipping.")
        return

    # 0) AOI grid from first file
    t0 = time.time()
    aoi_geom, dst_transform, width, height, resx, resy = build_aoi_grid(aoi_path, paths[0])
    mask_outside = ~geometry_mask([aoi_geom.__geo_interface__],
                                  out_shape=(height, width),
                                  transform=dst_transform,
                                  invert=True)
    stamp(f"AOI grid 2056: {width}x{height} px @ ~{resx:.3f} m (setup {time.time()-t0:.1f}s)")

    aoi_bounds_2056 = aoi_geom.bounds

    # Output dir (per year × method)
    out_dir  = os.path.dirname(paths[0])
    clip_dir = os.path.join(out_dir, f"_{year}_{label}_clip2056_ONLY")
    os.makedirs(clip_dir, exist_ok=True)

    ref_count = None
    ref_dtype = None
    t1_all = time.time()
    written = 0

    for idx, fp in enumerate(paths, 1):
        base = os.path.splitext(os.path.basename(fp))[0]
        out_clip = os.path.join(clip_dir, base + "_clip2056_ONLY_SINGLE.bsq")
        out_png  = os.path.join(clip_dir, base + "_clip2056_ONLY_RGB.png")
        t1 = time.time()

        with rasterio.open(fp) as src:
            src_crs, note = detect_effective_src_crs(src)
            if note != "[as-is]":
                stamp(f"[info] {os.path.basename(fp)} -> {note}")

            if not overlaps_aoi_in_2056(src.bounds, src_crs, aoi_bounds_2056):
                print(f"[skip] No AOI overlap (in 2056): {os.path.basename(fp)}")
                continue

            if ref_count is None:
                ref_count = src.count
                ref_dtype = src.dtypes[0]
            elif src.count != ref_count:
                raise ValueError(f"Band count mismatch: {os.path.basename(fp)} has {src.count}, expected {ref_count}")

            cleanup_sidecars(out_clip)

            profile = {
                "driver": "ENVI",
                "height": height,
                "width":  width,
                "count":  ref_count,
                "crs": TARGET_CRS,
                "transform": dst_transform,
                "dtype":  ref_dtype,
                "nodata": (src.nodata if src.nodata is not None else 0.0),
                "interleave": "bsq",
            }

            with rasterio.open(out_clip, "w", **profile) as dst:
                nod = profile["nodata"]
                for b in tqdm(range(1, ref_count + 1),
                              desc=f"Clip {base}", unit="band", leave=False):
                    dst_arr = np.full((height, width), nod, dtype=ref_dtype)
                    reproject(
                        source=src.read(b),
                        destination=dst_arr,
                        src_transform=src.transform,
                        src_crs=src_crs,
                        dst_transform=dst_transform,
                        dst_crs=TARGET_CRS,
                        src_nodata=(src.nodata if src.nodata is not None else 0.0),
                        dst_nodata=nod,
                        resampling=RESAMPLING,
                        init_dest_nodata=True,
                    )
                    dst_arr[mask_outside] = nod
                    dst.write(dst_arr, b)

        stamp(f"[{idx}/{len(paths)}] wrote: {os.path.basename(out_clip)} ({time.time()-t1:.1f}s)")
        written += 1

        # Optional quick RGB per file
        try:
            with rasterio.open(out_clip) as s:
                r_idx = min(39, s.count); g_idx = min(17, s.count); b_idx = min(6, s.count)
                r = s.read(r_idx); g = s.read(g_idx); b = s.read(b_idx)
            rgb = np.dstack((norm(r), norm(g), norm(b)))
            plt.figure(figsize=(7,7)); plt.imshow(rgb); plt.axis('off')
            plt.title(f"{label} {year} — {base} RGB (R39,G17,B6)")
            plt.savefig(out_png, dpi=180, bbox_inches="tight"); plt.close()
        except Exception as e:
            print(f"[note] RGB preview skipped for {base}: {e}")

        # Summary
        try:
            with rasterio.open(out_clip) as s:
                print("  └─ Summary:",
                      "CRS", s.crs, "| Bands", s.count,
                      "| Size", f"{s.width}x{s.height}", "| DType", s.dtypes[0])
        except Exception as e:
            print("  └─ Summary unavailable:", e)

    stamp(f"Clipping done in {time.time()-t1_all:.1f}s; files written: {written}")

# ===================== RUN ALL =====================
for label, years in CFG.items():
    for year, paths in years.items():
        process_one_set(label, year, paths, aoi_path=AOI_VECTOR)



Lambertian — 2010 (CLIP ONLY, NO MOSAIC)
[16:11:53] AOI grid 2056: 3709x3580 px @ ~2.000 m (setup 0.8s)
[16:11:53] [info] CHNP1100624_052e_polishRefl_sub.bsq -> [assumed: LV03 default]


[16:15:31] [1/1] wrote: CHNP1100624_052e_polishRefl_sub_clip2056_ONLY_SINGLE.bsq (218.2s)
  └─ Summary: CRS EPSG:2056 | Bands 285 | Size 3709x3580 | DType int16
[16:15:35] Clipping done in 221.7s; files written: 1

Lambertian — 2012 (CLIP ONLY, NO MOSAIC)
[16:15:35] AOI grid 2056: 3709x3580 px @ ~2.000 m (setup 0.3s)


[16:23:10] [1/1] wrote: CHNP2120629_a32d_polish_cube000_geo_s1_clip2056_ONLY_SINGLE.bsq (454.9s)
  └─ Summary: CRS EPSG:2056 | Bands 299 | Size 3709x3580 | DType int16
[16:23:14] Clipping done in 459.3s; files written: 1

Lambertian — 2013 (CLIP ONLY, NO MOSAIC)
[16:23:14] AOI grid 2056: 3709x3580 px @ ~2.000 m (setup 0.1s)
[16:23:14] [info] M0038130712_a032e_reflct_cube000_geo.bsq -> [assumed: LV03 default]


[16:38:10] [1/1] wrote: M0038130712_a032e_reflct_cube000_geo_clip2056_ONLY_SINGLE.bsq (895.6s)
  └─ Summary: CRS EPSG:2056 | Bands 284 | Size 3709x3580 | DType int16
[16:38:15] Clipping done in 900.3s; files written: 1
